In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
from datetime import datetime
from pathlib import Path
import sys

import matplotlib.pyplot as plt
from pymatgen.core.structure import Molecule

Custom plotting code... ignore this if you don't know what it is, it should gracefully do nothing if you don't have the `MPLAdjutant` class. 

In [ ]:
sys.path.append(str(Path.home() / Path("local")))
class NullClass:
    def do_nothing(*args, **kwargs):
        pass
    def add_colorbar(self, im, **kwargs):
        return plt.colorbar(im)
    def __getattr__(self, _):
        return self.do_nothing
try:
    from mpl_utils import MPLAdjutant
    adj = MPLAdjutant()
    adj.set_defaults()
except ImportError:
    adj = NullClass()

Append the `home` path of this project.

In [ ]:
sys.path.append(str(Path.cwd().parent))

# Introduction

This notebook contains the final post-processing scripts for preparing the machine learning (ML) inputs for the QM9 datasets. If you want to generate all files from scratch, you'll need to run the `XAS-NNE/data/qm9/00_process_qm9_data.py` script from within the `XAS-NNE/data/qm9/` directory.

However, we have constructed the ML inputs already and stored them in the GitHub repository. These inputs are all derived directly from the final output `pickle` file from `XAS-NNE/data/qm9/00_process_qm9_data.py`.

# Construct the ACSF feature vectors

We use the Atom-centered Symmetry Functions (ACSF) feature vectors as inputs for the ML models. See [here](https://singroup.github.io/dscribe/latest/tutorials/descriptors/acsf.html) for the `Dscribe` library implementation docs. This is the original paper:

> Jörg Behler. Atom-centered symmetry functions for constructing high-dimensional neural network potentials. J. Chem. Phys., 134(7):074106, 2011.

We use roughly the same parameters as in the original ænet paper:

> ...

In [ ]:
from dscribe.descriptors import ACSF

Read in the results from the outputs of the previous scripts. This is a relatively large `pickle` file (~20 GB) so proceed with caution.

In [ ]:
path = Path("../data/qm9/XANES-220622-C-N-O.pkl")
print(path.exists())
data = pickle.load(open(path, "rb"))

Setup some common grids for interpolating all of the spectra onto.

In [ ]:
N = 200
grids = {
    "O": np.linspace(528, 582, N),
    "N": np.linspace(395, 449, N),
    "C": np.linspace(275, 329, N)
}

## Construct for each absorbing atom type

In [ ]:
CENTRAL_ATOM = "C"   # C, N or O

In [ ]:
species = ["H", "C", "O", "N", "F"]
rcut = 6.0
g2_params = [[1.0, 0], [0.1, 0], [0.01, 0]]
g4_params=[
    [0.001, 1.0, -1.0],
    [0.001, 2.0, -1.0],
    [0.001, 4.0, -1.0],
    [0.01, 1.0, -1.0],
    [0.01, 2.0, -1.0],
    [0.01, 4.0, -1.0],
    [0.1, 1.0, -1.0],
    [0.1, 2.0, -1.0],
    [0.1, 3.0, -1.0]
]
grid = grids[CENTRAL_ATOM]
acsf = ACSF(
    species=species,
    rcut=rcut,
    g2_params=g2_params,
    g4_params=g4_params
)

Generate the ACSF vectors.

In [ ]:
origin_smiles = []
molecule_site_pairs = []
acsf_array = []
spectra = []

# cc = 0
for qm9id, datum in tqdm(data.items()):
    molecule = Molecule.from_dict(datum["molecule"])
    atoms = []
    central_atom_indexes = []
    for ii, site in enumerate(molecule):
        atom = Atom(site.specie.symbol, site.coords)
        atoms.append(atom)
        if site.specie.symbol == CENTRAL_ATOM:
            central_atom_indexes.append(ii)
    atoms = Atoms(atoms)
    
    if len(central_atom_indexes) == 0:
        continue

    tmp_acsf = acsf.create(atoms, positions=central_atom_indexes)
    
    for idx, ii in enumerate(central_atom_indexes):
        key = f"{ii}_{CENTRAL_ATOM}"
        s = np.array(datum["xanes"][key]["spectrum"])

        try:
            spline = InterpolatedUnivariateSpline(s[:, 0], s[:, 3])
        except IndexError:
            continue
            
        res = spline(grid)
        
        # Oxygen and Carbon screening condition for unphysical/outlier results
        if CENTRAL_ATOM == "O":
            if np.any(res[:10] > 5.0):
                continue
        elif CENTRAL_ATOM == "C":
            if np.any(res[:35] > 10.0):
                continue
        
        origin_smiles.append(datum["smiles"])
        spectra.append(res)
        acsf_array.append(tmp_acsf[idx])
        molecule_site_pairs.append(f"{qm9id}_{ii}")
    
#     cc += 1
    
#     if cc > 20:
#         break

acsf_array = np.array(acsf_array)
spectra = np.array(spectra)

## PCA for debugging

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
pca_acsf = PCA(2)
w_acsf = pca_acsf.fit_transform(acsf_array)

In [ ]:
pca_spectra = PCA(2)
w_spectra = pca_spectra.fit_transform(spectra)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 3))

ax.scatter(w_acsf[:, 0], w_acsf[:, 1], alpha=0.02, c=w_spectra[:, 0], s=1, cmap="rainbow")

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))

N = 100
M = 100

dig = np.digitize(w_acsf[::N, 0], bins=np.linspace(w_acsf[::N, 0].min(), w_acsf[::N, 0].max(), M))
cmap = cm.get_cmap("rainbow", M)

for ii, (spectrum, col, key) in enumerate(zip(spectra[::N], w_spectra[::N, 0], molecule_site_pairs[::N])):
    ax.plot(grid, spectrum, color=cmap(dig[ii]), alpha=0.1)

plt.show()

After confirming everything looks good, we save to disk.

In [ ]:
now = datetime.now().strftime("%y%m%d")
fname = f"../data/qm9/XANES-{now}-ACSF-{CENTRAL_ATOM}.pkl"
print(fname)

We take the convention that `"x"` is the input and `"y"` is the output. These are the only two required keys for the ML pipeline. The rest is considered metadata.

In [ ]:
pickle.dump(
    {"grid": grid, "y": spectra, "x": acsf_array, "names": molecule_site_pairs, "origin_smiles": origin_smiles},
    open(fname, "wb"),
    protocol=pickle.HIGHEST_PROTOCOL
)